In [1]:
import sys
print("python exe:", sys.executable)   # full path to the interpreter
print("python ver:", sys.version)

# try torch only in the working notebook
try:
    import torch
    print("torch      :", torch.__version__, torch.__file__)
except ModuleNotFoundError as e:
    print("torch not importable:", e)



python exe: c:\Users\aneek\anaconda3\envs\tf_gpu_env\python.exe
python ver: 3.9.23 | packaged by conda-forge | (main, Jun  4 2025, 17:49:16) [MSC v.1929 64 bit (AMD64)]
torch      : 1.12.1+cu113 c:\Users\aneek\anaconda3\envs\tf_gpu_env\lib\site-packages\torch\__init__.py


c:\Users\aneek\anaconda3\envs\tf_gpu_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tensorflow as tf
print(tf.__version__)  # This should print the version of TensorFlow
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

print("CUDA version:", tf.sysconfig.get_build_info()["cuda_version"])
print("cuDNN version:", tf.sysconfig.get_build_info()["cudnn_version"])

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2.10.0
Num GPUs Available:  1
CUDA version: 64_112
cuDNN version: 64_8
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2821328081990942938
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5713690624
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3825921087133055220
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


In [3]:
import torch
print(torch.__version__)
print("CUDA Available: ", torch.cuda.is_available())


1.12.1+cu113
CUDA Available:  True


In [4]:
import tensorflow as tf
print("TensorFlow devices:")
print(tf.config.list_physical_devices())

# Force TensorFlow to use GPU
tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[0], True)

# Test a simple matrix multiplication
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0], [3.0, 4.0]])
    b = tf.constant([[5.0, 6.0], [7.0, 8.0]])
    c = tf.matmul(a, b)
    print("TensorFlow running on GPU:", c)


TensorFlow devices:
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow running on GPU: tf.Tensor(
[[19. 22.]
 [43. 50.]], shape=(2, 2), dtype=float32)


In [4]:
import torch
print("Torch CUDA Version:", torch.version.cuda)
print("Torch cuDNN Version:", torch.backends.cudnn.version())
print("Available GPUs:", torch.cuda.device_count())
print("GPU Name:", torch.cuda.get_device_name(0))


Torch CUDA Version: 11.3
Torch cuDNN Version: 8302
Available GPUs: 1
GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU


In [5]:
import torch
print(f"Allocated GPU memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Reserved GPU memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")


Allocated GPU memory: 0.00 GB
Reserved GPU memory: 0.00 GB


In [7]:
import tensorflow as tf
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
import torch
torch.cuda.empty_cache()


In [6]:
import time
import tensorflow as tf
import psutil

class PowerMonitor:
    def __init__(self):
        self.gpu_available = tf.config.list_physical_devices('GPU')
        
        # Hardware power specifications (adjust these values for your system)
        self.cpu_tdp = 65    # Typical TDP for desktop CPUs in watts
        self.gpu_tdp = 250   # Typical TDP for desktop GPUs in watts
        
    def get_stats(self):
        """Get system stats with power estimation"""
        stats = {
            'timestamp': time.time(),
            'cpu_%': psutil.cpu_percent(interval=0.1),
            'ram_mb': psutil.virtual_memory().used / (1024**2),
            'gpu_mem_mb': 0,
            'power_w': self.cpu_tdp * (psutil.cpu_percent()/100) * 0.85  # Base CPU power
        }
        
        if self.gpu_available:
            try:
                # TensorFlow GPU memory monitoring
                mem_info = tf.config.experimental.get_memory_info('GPU:0')
                stats.update({
                    'gpu_mem_mb': mem_info['current'] / (1024**2),
                    'power_w': self.cpu_tdp * (psutil.cpu_percent()/100) * 0.85 + 
                              self.gpu_tdp * 0.5 * 0.75  # Add GPU power estimate
                })
            except:
                pass
                
        return stats

# Initialize monitor
monitor = PowerMonitor()

In [7]:
import time
import torch
import psutil
import os

class PowerMonitor1:
    def __init__(self):
        self.gpu_available = torch.cuda.is_available()
        self.process = psutil.Process(os.getpid())  # Track current process
        
        # Hardware power specifications
        self.cpu_tdp = 65
        self.gpu_tdp = 250
        
    def get_stats(self):
        """Get process-specific stats with power estimation"""
        process_memory = self.process.memory_info()
        
        stats = {
            'timestamp': time.time(),
            'cpu_%': psutil.cpu_percent(interval=0.1),
            'process_ram_mb': process_memory.rss / (1024**2),  # Only this process's RAM
            'gpu_mem_mb': 0,
            'power_w': self.cpu_tdp * (psutil.cpu_percent()/100) * 0.85
        }
        
        if self.gpu_available:
            try:
                gpu_memory_allocated = torch.cuda.memory_allocated()
                stats.update({
                    'gpu_mem_mb': gpu_memory_allocated / (1024**2),
                    'power_w': self.cpu_tdp * (psutil.cpu_percent()/100) * 0.85 + 
                              self.gpu_tdp * 0.5 * 0.75
                })
            except Exception as e:
                print(f"Error retrieving GPU memory: {e}")
                
        return stats

# Initialize monitor1
monitor1 = PowerMonitor1()

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


# Split the data into train (70%) and test (30%)
X_train_real, X_test_real = train_test_split(real_frames_array1, test_size=0.3, random_state=42)
X_train_fake, X_test_fake = train_test_split(fake_frames_array1, test_size=0.3, random_state=42)


# Output the shape to confirm the split dimensions
print("Training data real shape:", X_train_real.shape)
print("Testing real data shape:", X_test_real.shape)
print("Training fake data shape:", X_train_fake.shape)
print("Testing  fake data shape:", X_test_fake.shape)
########################################################################################################################################
#######################################divide into 60,10 train and val
#########################################################################################################################################
def extract_validation(train_data):
    """
    Extract every 10th sample from the training data and store it in a validation set.

    Parameters:
        train_data (list or np.array): The training dataset.

    Returns:
        tuple: Updated training dataset and validation dataset.
    """
    # Select every 10th sample for the validation set
    validation_data = train_data[::10]

    # Remove the selected samples from the training dataset
    updated_train_data = [train_data[i] for i in range(len(train_data)) if i % 10 != 0]

    return np.array(updated_train_data), np.array(validation_data)


# Perform the operation for each dataset
train_hog_real, val_hog_real = extract_validation(X_train_real)
train_hog_fake, val_hog_fake = extract_validation(X_train_fake)

# Print results for verification
print(f"train_hog_real: {len(train_hog_real)} images, val_hog_real: {len(val_hog_real)} images")
print(f"train_hog_fake: {len(train_hog_fake)} images, val_hog_fake: {len(val_hog_fake)} images")


############################################################################################################################################################
#################################################concatenate the labels 0,1 real and fake
#############################################################################################################################################################


train_labels_real = np.zeros(len(train_hog_real), dtype=int)
train_labels_fake = np.ones(len(train_hog_fake), dtype=int)

# Concatenate all training datasets into a single `train` variable
train = np.concatenate([train_hog_real, train_hog_fake], axis=0)
train_labels=np.concatenate([train_labels_real, train_labels_fake], axis=0)



test_labels_real = np.zeros(len(X_test_real), dtype=int)
test_labels_fake = np.ones(len(X_train_fake), dtype=int)

# Concatenate all testing datasets into a single `test` variable
test = np.concatenate([X_test_real, X_train_fake], axis=0)
test_labels = np.concatenate([test_labels_real, test_labels_fake], axis=0)



val_labels_real = np.zeros(len(val_hog_real), dtype=int)
val_labels_fake = np.ones(len(val_hog_fake), dtype=int)

# Concatenate all validation datasets into a single `val` variable
val = np.concatenate([val_hog_real, val_hog_fake], axis=0)
val_labels = np.concatenate([val_labels_real, val_labels_fake], axis=0)

# Print the results for verification
print(f"Total train: {len(train)} images")
print(f"Total test: {len(test)} images")
print(f"Total val: {len(val)} images")


# Print results for verification
print(f"Train Labels: {len(train_labels)} ")
print(f"Test Labels: {len(test_labels)} ")
print(f"Val Labels: {len(val_labels)} ")



Training data real shape: (6909, 160, 160, 3)
Testing real data shape: (2961, 160, 160, 3)
Training fake data shape: (7891, 160, 160, 3)
Testing  fake data shape: (3383, 160, 160, 3)
train_hog_real: 6218 images, val_hog_real: 691 images
train_hog_fake: 7101 images, val_hog_fake: 790 images
Total train: 13319 images
Total test: 10852 images
Total val: 1481 images
Train Labels: 13319 
Test Labels: 10852 
Val Labels: 1481 


In [5]:
print("=== DATA LOADING ===")
start = monitor.get_stats()

=== DATA LOADING ===


In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from PIL import Image
from tqdm import tqdm

# 1. Load MaxViT Model
model_name = "maxvit_base_tf_224"  # Options: maxvit_tiny_tf_224, maxvit_small_tf_224, etc.
model = timm.create_model(model_name, pretrained=True, num_classes=2)  # Set your num_classes
config = resolve_data_config({}, model=model)
transform = create_transform(**config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2. Custom Dataset (same as before)
class ImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if isinstance(image, np.ndarray):
            image = Image.fromarray(image)
        elif isinstance(image, str):
            image = Image.open(image).convert("RGB")

        if self.transform:
            image = self.transform(image)
            
        return image, torch.tensor(label, dtype=torch.long)

In [9]:
import torch.optim as optim  # This is the missing 
import torch.nn as nn
from torch.optim.lr_scheduler import LinearLR, CosineAnnealingLR  # Add these imports
# 3. Training Function
def train_model(model, train_loader, val_loader, epochs=10):
    optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
    criterion = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler()
    
    # Warmup + cosine scheduling
    scheduler = torch.optim.lr_scheduler.SequentialLR(
        optimizer,
        [
            LinearLR(optimizer, start_factor=0.01, total_iters=3),
            CosineAnnealingLR(optimizer, T_max=epochs-3)
        ],
        milestones=[3]
    )
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device, non_blocking=True), labels.to(device)
            
            optimizer.zero_grad()
            
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        scheduler.step()
        train_acc = 100 * correct / total
        val_metrics = evaluate_metrics(model, val_loader)
        print(f"Epoch {epoch+1}: LR = {optimizer.param_groups[0]['lr']:.2e}, Loss = {train_loss/len(train_loader):.4f}, Acc = {train_acc:.2f}%")
        print(f"Val Metrics: {val_metrics}\n")

# 4. Evaluation Function
def evaluate_metrics(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            preds = outputs.argmax(1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return {
        'accuracy': accuracy_score(all_labels, all_preds),
        'precision': precision_score(all_labels, all_preds, average='weighted'),
        'recall': recall_score(all_labels, all_preds, average='weighted'),
        'f1': f1_score(all_labels, all_preds, average='weighted')
    }

In [7]:
# Convert data into PyTorch Dataset
train_dataset = ImageDataset(train, train_labels, transform)
val_dataset = ImageDataset(val, val_labels, transform)
test_dataset = ImageDataset(test, test_labels, transform)

# Dataloaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

NameError: name 'ImageDataset' is not defined

In [16]:
# Train the model
epochs = 10  # Increase epochs for better accuracy
train_model(model, train_loader, val_loader, epochs)

Epoch 1/10: 100%|██████████| 833/833 [4:51:39<00:00, 21.01s/it]  


Epoch 1: LR = 3.40e-05, Loss = 0.6930, Acc = 52.59%
Val Metrics: {'accuracy': 0.5665091154625254, 'precision': 0.5630607328757162, 'recall': 0.5665091154625254, 'f1': 0.5573460582633291}



Epoch 2/10: 100%|██████████| 833/833 [4:59:47<00:00, 21.59s/it]  


Epoch 2: LR = 6.70e-05, Loss = 0.2765, Acc = 87.33%
Val Metrics: {'accuracy': 0.9392302498311952, 'precision': 0.9440657849220603, 'recall': 0.9392302498311952, 'f1': 0.9388276149495407}



Epoch 3/10: 100%|██████████| 833/833 [5:03:26<00:00, 21.86s/it]  
c:\Users\aneek\anaconda3\envs\tf_gpu_env\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


...

Epoch 8: LR = 1.88e-05, Loss = 0.0064, Acc = 99.78%
Val Metrics: {'accuracy': 0.9736664415935179, 'precision': 0.9745708286455621, 'recall': 0.9736664415935179, 'f1': 0.9736918037732227}



Epoch 9/10: 100%|██████████| 833/833 [5:02:36<00:00, 21.80s/it]  


Epoch 9: LR = 4.95e-06, Loss = 0.0013, Acc = 99.97%
Val Metrics: {'accuracy': 0.9891964888588791, 'precision': 0.989222902736277, 'recall': 0.9891964888588791, 'f1': 0.989193370085226}



Epoch 10/10: 100%|██████████| 833/833 [4:53:06<00:00, 21.11s/it]  


Epoch 10: LR = 0.00e+00, Loss = 0.0006, Acc = 99.98%
Val Metrics: {'accuracy': 0.9891964888588791, 'precision': 0.9891981147666367, 'recall': 0.9891964888588791, 'f1': 0.9891954892485665}



In [12]:
print("=== DATA LOADING ===")
start = monitor1.get_stats()
# Final evaluation on test data
test_metrics = evaluate_metrics(model, test_loader)
# Your data loading operations here
time.sleep(2)  # Simulate loading time
#7.7 gpu

end = monitor1.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")#7.7

=== DATA LOADING ===

=== RESOURCE USAGE ===
CPU Usage: 2.1%
Time Usage: 868.4 s
GPU Memory Used: 929.1 MB


In [13]:
start = monitor.get_stats()
test_metrics = evaluate_metrics(model, test_loader)
end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")


RAM Used: 16.2 MB


In [18]:
test_metrics = evaluate_metrics(model, test_loader)

In [19]:
print(f"Test Metrics:")
print(f"- Accuracy: {test_metrics['accuracy']:.4f}")
print(f"- Precision: {test_metrics['precision']:.4f}")
print(f"- Recall: {test_metrics['recall']:.4f}")
print(f"- F1 Score: {test_metrics['f1']:.4f}")

Test Metrics:
- Accuracy: 0.9963
- Precision: 0.9963
- Recall: 0.9963
- F1 Score: 0.9963


In [ ]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time
#7.8 gpu

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts


=== RESOURCE USAGE ===
CPU Usage: 2.7%
RAM Used: 7636.5 MB
Time Usage: 186467.5 s
GPU Memory Used: 0.0 MB
Power Consumption: 93W


In [21]:

test_dataset = ImageDataset(test_hog, test_labels, transform)
# Dataloaders
batch_size = 16
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f"Test Accuracy of DFC on Celeb DF (v2)")
# Final evaluation on test data
test_metrics = evaluate_metrics(model, test_loader)
print(f"Test Metrics:")
print(f"- Accuracy: {test_metrics['accuracy']:.4f}")
print(f"- Precision: {test_metrics['precision']:.4f}")
print(f"- Recall: {test_metrics['recall']:.4f}")
print(f"- F1 Score: {test_metrics['f1']:.4f}")
#print(f"Test Accuracy of DFC on Celeb DF (v2): {test_accuracy:.4f}")

Test Accuracy of DFC on Celeb DF (v2)
Test Metrics:
- Accuracy: 0.4947
- Precision: 0.4561
- Recall: 0.4947
- F1 Score: 0.3524


In [23]:

test_dataset = ImageDataset(test_ff, test_labels_ff, transform)
# Dataloaders
batch_size = 16
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f"Test Accuracy of FF++ on Celeb DF (v2)")
# Final evaluation on test data
test_metrics = evaluate_metrics(model, test_loader)
print(f"Test Metrics:")
print(f"- Accuracy: {test_metrics['accuracy']:.4f}")
print(f"- Precision: {test_metrics['precision']:.4f}")
print(f"- Recall: {test_metrics['recall']:.4f}")
print(f"- F1 Score: {test_metrics['f1']:.4f}")


Test Accuracy of FF++ on Celeb DF (v2)
Test Metrics:
- Accuracy: 0.3813
- Precision: 0.6900
- Recall: 0.3813
- F1 Score: 0.3412


# DFC

In [8]:
import h5py
import numpy as np
# Open the HDF5 file in read mode
with h5py.File('D://thesis//dataset//deepfake dataset//resized_images.h5', 'r') as h5f:
    # Access each dataset
    celeb = np.array(h5f['celeb'])
    ffhq = np.array(h5f['ffhq'])
    gdwct = np.array(h5f['gdwct'])
    attgan = np.array(h5f['attgan'])
    stargan = np.array(h5f['stargan'])
    stylegan2 = np.array(h5f['stylegan2'])
    stylegan = np.array(h5f['stylegan'])

# Now, 'celeb', 'ffhq', etc., are NumPy arrays containing your datasets
print(f"celeb shape: {celeb.shape}, dtype: {celeb.dtype}")
print(f"ffhq shape: {ffhq.shape}, dtype: {ffhq.dtype}")
print(f"ffhq shape: {gdwct.shape}, dtype: {gdwct.dtype}")
print(f"ffhq shape: {attgan.shape}, dtype: {attgan.dtype}")
print(f"ffhq shape: {stargan.shape}, dtype: {stargan.dtype}")
print(f"ffhq shape: {stylegan2.shape}, dtype: {stylegan2.dtype}")
print(f"ffhq shape: {stylegan.shape}, dtype: {stylegan.dtype}")
# Repeat for other datasets as needed
import cv2
# Function to resize images from (224, 224) to (160, 160)
def resize_images(image_array, target_size=(160, 160)):
    resized_images = np.array([cv2.resize(img, target_size) for img in image_array])
    return resized_images

celeb = resize_images(celeb, target_size=(160, 160))
ffhq = resize_images(ffhq, target_size=(160, 160))
gdwct = resize_images(gdwct, target_size=(160, 160))
attgan = resize_images(attgan, target_size=(160, 160))
stargan = resize_images(stargan, target_size=(160, 160))
stylegan = resize_images(stargan, target_size=(160, 160))
stylegan2 = resize_images(stylegan2, target_size=(160, 160))
import random
# Randomly select 2500 distinct images
random_indices = random.sample(range(len(celeb)), 2500)  # Get 2500 random indices
celeb = celeb[random_indices]  # Select the random subse

import random
# Randomly select 2500 distinct images
random_indices = random.sample(range(len(ffhq)), 2500)  # Get 2500 random indices
ffhq = ffhq[random_indices]  # Select the random subse
print(f"celeb shape: {celeb.shape}, dtype: {celeb.dtype}")
print(f"ffhq shape: {ffhq.shape}, dtype: {ffhq.dtype}")
print(f"gdwct shape: {gdwct.shape}, dtype: {gdwct.dtype}")
print(f"attagan shape: {attgan.shape}, dtype: {attgan.dtype}")
print(f"stargan shape: {stargan.shape}, dtype: {stargan.dtype}")
print(f"stylegan2 shape: {stylegan2.shape}, dtype: {stylegan2.dtype}")
print(f"stylegan shape: {stylegan.shape}, dtype: {stylegan.dtype}")
import random
import numpy as np

def split_data(data, train_ratio=0.7):
    """
    Splits data into training and testing sets based on the specified ratio.

    Parameters:
        data (list or np.array): The dataset to split.
        train_ratio (float): The ratio of the data to include in the training set.

    Returns:
        tuple: Two datasets - train and test.
    """
    # Shuffle the data
    random.shuffle(data)

    # Calculate the split index
    split_index = int(len(data) * train_ratio)

    # Split the data
    train_data = data[:split_index]
    test_data = data[split_index:]

    return train_data, test_data

# Split `celeb` into 70% train and 30% test
celeb_train_hog, celeb_test_hog = split_data(celeb, train_ratio=0.7)

# Split `ffhq` into 70% train and 30% test
ffhq_train_hog, ffhq_test_hog = split_data(ffhq, train_ratio=0.7)

# Split `attgan` into 70% train and 30% test
attgan_train_hog, attgan_test_hog = split_data(attgan, train_ratio=0.7)

# Split `stargan` into 70% train and 30% test
stargan_train_hog, stargan_test_hog = split_data(stargan, train_ratio=0.7)

# Split `gdwct` into 70% train and 30% test
gdwct_train_hog, gdwct_test_hog = split_data(gdwct, train_ratio=0.7)

# Split `stylegan2` into 70% train and 30% test_hog
stylegan2_train_hog, stylegan2_test_hog = split_data(stylegan2, train_ratio=0.7)

# Split `stylegan` into 70% train and 30% test_hog
stylegan_train_hog, stylegan_test_hog = split_data(stylegan, train_ratio=0.7)

# Convert to NumPy arrays if needed
celeb_train_hog, celeb_test_hog = np.array(celeb_train_hog), np.array(celeb_test_hog)
ffhq_train_hog, ffhq_test_hog = np.array(ffhq_train_hog), np.array(ffhq_test_hog)
attgan_train_hog, attgan_test_hog = np.array(attgan_train_hog), np.array(attgan_test_hog)
stargan_train_hog, stargan_test_hog = np.array(stargan_train_hog), np.array(stargan_test_hog)
gdwct_train_hog, gdwct_test_hog = np.array(gdwct_train_hog), np.array(gdwct_test_hog)
stylegan2_train_hog, stylegan2_test_hog = np.array(stylegan2_train_hog), np.array(stylegan2_test_hog)
stylegan_train_hog, stylegan_test_hog = np.array(stylegan_train_hog), np.array(stylegan_test_hog)

# Print results for verification
print(f"celeb_train: {len(celeb_train_hog)} images, celeb_test: {len(celeb_test_hog)} images")
print(f"ffhq_train: {len(ffhq_train_hog)} images, ffhq_test: {len(ffhq_test_hog)} images")
print(f"attgan_train: {len(attgan_train_hog)} images, attgan_test: {len(attgan_test_hog)} images")
print(f"stargan_train: {len(stargan_train_hog)} images, stargan_test: {len(stargan_test_hog)} images")
print(f"gdwct_train: {len(gdwct_train_hog)} images, gdwct_test: {len(gdwct_test_hog)} images")
print(f"stylegan2_train: {len(stylegan2_train_hog)} images, stylegan2_test: {len(stylegan2_test_hog)} images")
print(f"stylegan_train: {len(stylegan_train_hog)} images, stylegan_test: {len(stylegan_test_hog)} images")

########################################################################################################################################
#######################################divide into 60,10 train and val
#########################################################################################################################################
def extract_validation(train_data):
    """
    Extract every 10th sample from the training data and store it in a validation set.

    Parameters:
        train_data (list or np.array): The training dataset.

    Returns:
        tuple: Updated training dataset and validation dataset.
    """
    # Select every 10th sample for the validation set
    validation_data = train_data[::10]

    # Remove the selected samples from the training dataset
    updated_train_data = [train_data[i] for i in range(len(train_data)) if i % 10 != 0]

    return np.array(updated_train_data), np.array(validation_data)


# Perform the operation for each dataset
celeb_train_hog, celeb_val_hog = extract_validation(celeb_train_hog)
ffhq_train_hog, ffhq_val_hog = extract_validation(ffhq_train_hog)
attgan_train_hog, attgan_val_hog = extract_validation(attgan_train_hog)
stargan_train_hog, stargan_val_hog = extract_validation(stargan_train_hog)
gdwct_train_hog, gdwct_val_hog = extract_validation(gdwct_train_hog)
stylegan2_train_hog, stylegan2_val_hog = extract_validation(stylegan2_train_hog)
stylegan_train_hog, stylegan_val_hog = extract_validation(stylegan_train_hog)

# Print results for verification
print(f"celeb_train: {len(celeb_train_hog)} images, celeb_val: {len(celeb_val_hog)} images")
print(f"ffhq_train: {len(ffhq_train_hog)} images, ffhq_val: {len(ffhq_val_hog)} images")
print(f"attgan_train: {len(attgan_train_hog)} images, attgan_val: {len(attgan_val_hog)} images")
print(f"stargan_train: {len(stargan_train_hog)} images, stargan_val: {len(stargan_val_hog)} images")
print(f"gdwct_train: {len(gdwct_train_hog)} images, gdwct_val: {len(gdwct_val_hog)} images")
print(f"stylegan2_train: {len(stylegan2_train_hog)} images, stylegan2_val: {len(stylegan2_val_hog)} images")
print(f"stylegan_train: {len(stylegan_train_hog)} images, stylegan_val: {len(stylegan_val_hog)} images")
############################################################################################################################################################
#################################################concatenate the labels 0,1 real and fake
#############################################################################################################################################################


celeb_train_labels = np.zeros(len(celeb_train_hog), dtype=int)
ffhq_train_labels = np.zeros(len(ffhq_train_hog), dtype=int)
atta_train_labels = np.ones(len(attgan_train_hog), dtype=int)
star_train_labels = np.ones(len(stargan_train_hog), dtype=int)
gdwct_train_labels = np.ones(len(gdwct_train_hog), dtype=int)
stylegan2_train_labels = np.ones(len(stylegan2_train_hog), dtype=int)
stylegan_train_labels = np.ones(len(stylegan_train_hog), dtype=int)

# Concatenate all training datasets into a single `train` variable
train_hog = np.concatenate([celeb_train_hog, ffhq_train_hog, attgan_train_hog, stargan_train_hog, gdwct_train_hog, stylegan2_train_hog, stylegan_train_hog], axis=0)
train_labels=np.concatenate([celeb_train_labels, ffhq_train_labels, atta_train_labels, star_train_labels, gdwct_train_labels, stylegan2_train_labels,
                              stylegan_train_labels], axis=0)




celeb_test_labels = np.zeros(len(celeb_test_hog), dtype=int)
ffhq_test_labels = np.zeros(len(ffhq_test_hog), dtype=int)
atta_test_labels = np.ones(len(attgan_test_hog), dtype=int)
star_test_labels = np.ones(len(stargan_test_hog), dtype=int)
gdwct_test_labels = np.ones(len(gdwct_test_hog), dtype=int)
stylegan2_test_labels = np.ones(len(stylegan2_test_hog), dtype=int)
stylegan_test_labels = np.ones(len(stylegan_test_hog), dtype=int)

# Concatenate all testing datasets into a single `test` variable
test_hog = np.concatenate([celeb_test_hog, ffhq_test_hog, attgan_test_hog, stargan_test_hog, gdwct_test_hog, stylegan2_test_hog, stylegan_test_hog], axis=0)
test_labels = np.concatenate([celeb_test_labels, ffhq_test_labels, atta_test_labels, star_test_labels, gdwct_test_labels, stylegan2_test_labels,
                        stylegan_test_labels], axis=0)




celeb_val_labels = np.zeros(len(celeb_val_hog), dtype=int)
ffhq_val_labels = np.zeros(len(ffhq_val_hog), dtype=int)
atta_val_labels = np.ones(len(attgan_val_hog), dtype=int)
star_val_labels = np.ones(len(stargan_val_hog), dtype=int)
gdwct_val_labels = np.ones(len(gdwct_val_hog), dtype=int)
stylegan2_val_labels = np.ones(len(stylegan2_val_hog), dtype=int)
stylegan_val_labels = np.ones(len(stylegan_val_hog), dtype=int)

# Concatenate all validation datasets into a single `val` variable
val_hog = np.concatenate([celeb_val_hog, ffhq_val_hog, attgan_val_hog, stargan_val_hog, gdwct_val_hog, stylegan2_val_hog, stylegan_val_hog], axis=0)
val_labels = np.concatenate([celeb_val_labels, ffhq_val_labels, atta_val_labels, star_val_labels, gdwct_val_labels, stylegan2_val_labels,
                       stylegan_val_labels], axis=0)

# Print the results for verification
print(f"Total train: {len(train_hog)} images")
print(f"Total test: {len(test_hog)} images")
print(f"Total val: {len(val_hog)} images")


# Print results for verification
print(f"Train Labels: {len(train_labels)} ")
print(f"Test Labels: {len(test_labels)} ")
print(f"Val Labels: {len(val_labels)} ")



celeb shape: (5000, 224, 224, 3), dtype: uint8
ffhq shape: (5000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
ffhq shape: (1000, 224, 224, 3), dtype: uint8
celeb shape: (2500, 160, 160, 3), dtype: uint8
ffhq shape: (2500, 160, 160, 3), dtype: uint8
gdwct shape: (1000, 160, 160, 3), dtype: uint8
attagan shape: (1000, 160, 160, 3), dtype: uint8
stargan shape: (1000, 160, 160, 3), dtype: uint8
stylegan2 shape: (1000, 160, 160, 3), dtype: uint8
stylegan shape: (1000, 160, 160, 3), dtype: uint8
celeb_train: 1750 images, celeb_test: 750 images
ffhq_train: 1750 images, ffhq_test: 750 images
attgan_train: 700 images, attgan_test: 300 images
stargan_train: 700 images, stargan_test: 300 images
gdwct_train: 700 images, gdwct_test: 300 images
stylegan2_train: 700 images, stylegan2_test: 300 images
stylegan_train: 700 images, stylegan

In [9]:
# Convert data into PyTorch Dataset
train_dataset = ImageDataset(train_hog, train_labels, transform)
val_dataset = ImageDataset(val_hog, val_labels, transform)
test_dataset = ImageDataset(test_hog, test_labels, transform)


# Dataloaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [12]:
# Train the model
epochs = 10  # Increase epochs for better accuracy
train_model(model, train_loader, val_loader, epochs)


Epoch 1/10: 100%|██████████| 394/394 [42:18<00:00,  6.44s/it]


Epoch 1: LR = 3.40e-05, Loss = 0.6811, Acc = 56.37%
Val Metrics: {'accuracy': 0.6728571428571428, 'precision': 0.6937771676961794, 'recall': 0.6728571428571428, 'f1': 0.663782706727492}



Epoch 2/10: 100%|██████████| 394/394 [41:58<00:00,  6.39s/it]


Epoch 2: LR = 6.70e-05, Loss = 0.1202, Acc = 95.79%
Val Metrics: {'accuracy': 0.9871428571428571, 'precision': 0.9872422943457848, 'recall': 0.9871428571428571, 'f1': 0.9871422011327107}



Epoch 3/10: 100%|██████████| 394/394 [45:45<00:00,  6.97s/it]
c:\Users\aneek\anaconda3\envs\tf_gpu_env\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


...

Epoch 8: LR = 1.88e-05, Loss = 0.0008, Acc = 100.00%
Val Metrics: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}



Epoch 9/10: 100%|██████████| 394/394 [45:44<00:00,  6.97s/it]


Epoch 9: LR = 4.95e-06, Loss = 0.0012, Acc = 99.97%
Val Metrics: {'accuracy': 0.9985714285714286, 'precision': 0.9985754985754985, 'recall': 0.9985714285714286, 'f1': 0.9985714256559707}



Epoch 10/10: 100%|██████████| 394/394 [45:45<00:00,  6.97s/it]


Epoch 10: LR = 0.00e+00, Loss = 0.0002, Acc = 100.00%
Val Metrics: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}



In [11]:
print("=== DATA LOADING ===")
start = monitor1.get_stats()
# Final evaluation on test data
test_metrics = evaluate_metrics(model, test_loader)
# Your data loading operations here
time.sleep(2)  # Simulate loading time
#7.7 gpu

end = monitor1.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")#7.7

=== DATA LOADING ===

=== RESOURCE USAGE ===
CPU Usage: 3.6%
Time Usage: 242.6 s
GPU Memory Used: 929.1 MB


In [13]:

# Final evaluation on test data
test_metrics = evaluate_metrics(model, test_loader)
print(f"Test Metrics:")
print(f"- Accuracy: {test_metrics['accuracy']:.4f}")
print(f"- Precision: {test_metrics['precision']:.4f}")
print(f"- Recall: {test_metrics['recall']:.4f}")
print(f"- F1 Score: {test_metrics['f1']:.4f}")

Test Metrics:
- Accuracy: 0.9950
- Precision: 0.9950
- Recall: 0.9950
- F1 Score: 0.9950


In [19]:

test_dataset = ImageDataset(test, test_labels, transform)
# Dataloaders
batch_size = 16
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f"Test Accuracy of Celeb DF(V2) on DFC")
# Final evaluation on test data
test_metrics = evaluate_metrics(model, test_loader)
print(f"Test Metrics:")
print(f"- Accuracy: {test_metrics['accuracy']:.4f}")
print(f"- Precision: {test_metrics['precision']:.4f}")
print(f"- Recall: {test_metrics['recall']:.4f}")
print(f"- F1 Score: {test_metrics['f1']:.4f}")

Test Accuracy of Celeb DF(V2) on DFC
Test Metrics:
- Accuracy: 0.7128
- Precision: 0.5969
- Recall: 0.7128
- F1 Score: 0.6173


In [16]:

test_dataset = ImageDataset(test_ff, test_labels_ff, transform)
# Dataloaders
batch_size = 16
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f"Test Accuracy of FF++ on DFC:")
# Final evaluation on test data
test_metrics = evaluate_metrics(model, test_loader)
print(f"Test Metrics:")
print(f"- Accuracy: {test_metrics['accuracy']:.4f}")
print(f"- Precision: {test_metrics['precision']:.4f}")
print(f"- Recall: {test_metrics['recall']:.4f}")
print(f"- F1 Score: {test_metrics['f1']:.4f}")

Test Accuracy of FF++ on DFC:
Test Metrics:
- Accuracy: 0.7211
- Precision: 0.6602
- Recall: 0.7211
- F1 Score: 0.6504


# FF++

In [10]:
import h5py

# Open the HDF5 file for reading
with h5py.File("D://thesis//dataset//ff++//images_ff++_orignal_processed.h5", "r") as h5f:
    org_seq_actor_array = h5f["ori_actor"][:]
    org_seq_youtube_array = h5f["ori_youtube"][:]
print("Data loaded successfully from the HDF5 file.")
with h5py.File("D://thesis//dataset//ff++//images_ff++_fake1_processed.h5", "r") as h5f:
    meni_seq_dfd_array = h5f["mni_dfd"][:]
    meni_seq_df_array = h5f["mni_df"][:]
print("Data loaded successfully from the HDF5 file.")
with h5py.File("D://thesis//dataset//ff++//images_ff++_fake2_processed.h5", "r") as h5f:
    meni_seq_f2f_array = h5f["mni_f2f"][:]
    meni_seq_fshifter_array = h5f["mni_fshifter"][:]
print("Data loaded successfully from the HDF5 file.")
with h5py.File("D://thesis//dataset//ff++//images_ff++_fake3_processed.h5", "r") as h5f:
    meni_seq_fswap_array = h5f["mni_fswap"][:]
    meni_seq_nt_array = h5f["mni_nt"][:]
print("Data loaded successfully from the HDF5 file.")
# Output the shape of the resulting array
print(f"Shape of the concatenated array: {org_seq_actor_array.shape}")
print(f"Shape of the concatenated array: {org_seq_youtube_array.shape}")

print(f"Shape of the concatenated array: {meni_seq_dfd_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_df_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_fshifter_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_f2f_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_fswap_array.shape}")
print(f"Shape of the concatenated array: {meni_seq_nt_array.shape}")
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

ori_actor_train_real, ori_actor_test_real = train_test_split(org_seq_actor_array, test_size=0.3, random_state=42)
ori_youtube_train_real, ori_youtube_test_real = train_test_split(org_seq_youtube_array, test_size=0.3, random_state=42)
# Split the data into train (70%) and test (30%)
mni_dfd_train_fake, mni_dfd_test_fake = train_test_split(meni_seq_dfd_array, test_size=0.3, random_state=42)
mni_df_train_fake, mni_df_test_fake = train_test_split(meni_seq_df_array, test_size=0.3, random_state=42)
mni_f2f_train_fake, mni_f2f_test_fake = train_test_split(meni_seq_f2f_array, test_size=0.3, random_state=42)
mni_fshifter_train_fake, mni_fshifter_test_fake = train_test_split(meni_seq_fshifter_array, test_size=0.3, random_state=42)
mni_fswap_train_fake, mni_fswap_test_fake = train_test_split(meni_seq_fswap_array, test_size=0.3, random_state=42)
mni_nt_train_fake, mni_nt_test_fake = train_test_split(meni_seq_nt_array, test_size=0.3, random_state=42)


# Output the shape to confirm the split dimensions
print("Training data ACTOR real shape:", ori_actor_train_real.shape, "Testing real data shape:", ori_actor_test_real.shape)
print("Training data Youtube real shape:", ori_youtube_train_real.shape, "Testing real data shape:", ori_youtube_test_real.shape)

print("Training data DFD fake shape:", mni_dfd_train_fake.shape, "Testing fake data shape:", mni_dfd_test_fake.shape)
print("Training data DF fake shape:", mni_df_train_fake.shape, "Testing fake data shape:", mni_df_test_fake.shape)
print("Training data f2f fake shape:", mni_f2f_train_fake.shape, "Testing fake data shape:", mni_f2f_test_fake.shape)
print("Training data fshifter fake shape:", mni_fshifter_train_fake.shape, "Testing fake data shape:", mni_fshifter_test_fake.shape)
print("Training data fswap fake shape:", mni_fswap_train_fake.shape, "Testing fake data shape:", mni_fswap_test_fake.shape)
print("Training data nt fake shape:", mni_nt_train_fake.shape, "Testing fake data shape:", mni_nt_test_fake.shape)
########################################################################################################################################
#######################################divide into 60,10 train and val
#########################################################################################################################################
def extract_validation(train_data):
    """
    Extract every 10th sample from the training data and store it in a validation set.

    Parameters:
        train_data (list or np.array): The training dataset.

    Returns:
        tuple: Updated training dataset and validation dataset.
    """
    # Select every 10th sample for the validation set
    validation_data = train_data[::10]

    # Remove the selected samples from the training dataset
    updated_train_data = [train_data[i] for i in range(len(train_data)) if i % 10 != 0]

    return np.array(updated_train_data), np.array(validation_data)


# Perform the operation for each dataset
train_hog_ori_actor_real, val_hog_ori_actor_real = extract_validation(ori_actor_train_real)
train_hog_ori_youtube_real, val_hog_ori_youtube_real = extract_validation(ori_youtube_train_real)
train_hog_mni_dfd_fake, val_hog_mni_dfd_fake = extract_validation(mni_dfd_train_fake)
train_hog_mni_df_fake, val_hog_mni_df_fake = extract_validation(mni_df_train_fake)
train_hog_mni_f2f_fake, val_hog_mni_f2f_fake = extract_validation(mni_f2f_train_fake)
train_hog_mni_fshifter_fake, val_hog_mni_fshifter_fake = extract_validation(mni_fshifter_train_fake)
train_hog_mni_fswap_fake, val_hog_mni_fswap_fake = extract_validation(mni_fswap_train_fake)
train_hog_mni_nt_fake, val_hog_mni_nt_fake = extract_validation(mni_nt_train_fake)
# Print results for verification
print(f"train_ori_actor hog_real: {len(train_hog_ori_actor_real)} images, val_ori_actor hog_real: {len(val_hog_ori_actor_real)} images")
print(f"train_ ori_youtube hog_real: {len(train_hog_ori_youtube_real)} images, val_ ori_youtube hog_real: {len(val_hog_ori_youtube_real)} images")
print(f"train_hog_mni_dfd_fake: {len(train_hog_mni_dfd_fake)} images, val_hog_mni_dfd_fake: {len(val_hog_mni_dfd_fake)} images")
print(f"train_hog_mni_df_fake: {len(train_hog_mni_df_fake)} images, val_hog_mni_df_fake: {len(val_hog_mni_df_fake)} images")
print(f"train_hog_mni_f2f_fake: {len(train_hog_mni_f2f_fake)} images, val_hog_mni_f2f_fake: {len(val_hog_mni_f2f_fake)} images")
print(f"train_hog_mni_fshifter_fake: {len(train_hog_mni_fshifter_fake)} images, val_hog_mni_fshifter_fake: {len(val_hog_mni_fshifter_fake)} images")
print(f"train_hog_mni_fswap_fake: {len(train_hog_mni_fswap_fake)} images, val_hog_mni_fswap_fake: {len(val_hog_mni_fswap_fake)} images")
print(f"train_hog_mni_nt_fake: {len(train_hog_mni_nt_fake)} images, val_hog_mni_nt_fake: {len(val_hog_mni_nt_fake)} images")
############################################################################################################################################################
#################################################concatenate the labels 0,1 real and fake
#############################################################################################################################################################


train_ori_actor_labels_real = np.zeros(len(train_hog_ori_actor_real), dtype=int)
train_ori_youtube_labels_real = np.zeros(len(train_hog_ori_youtube_real), dtype=int)
train_mni_dfd_labels_fake = np.ones(len(train_hog_mni_dfd_fake), dtype=int)
train_mni_df_labels_fake = np.ones(len(train_hog_mni_df_fake), dtype=int)
train_mni_f2f_labels_fake = np.ones(len(train_hog_mni_f2f_fake), dtype=int)
train_mni_fshifter_labels_fake = np.ones(len(train_hog_mni_fshifter_fake), dtype=int)
train_mni_fswap_labels_fake = np.ones(len(train_hog_mni_fswap_fake), dtype=int)
train_mni_nt_labels_fake = np.ones(len(train_hog_mni_nt_fake), dtype=int)

test_ori_actor_labels_real = np.zeros(len(ori_actor_test_real), dtype=int)
test_ori_youtube_labels_real = np.zeros(len(ori_youtube_test_real), dtype=int)
test_mni_dfd_labels_fake = np.ones(len(mni_dfd_test_fake), dtype=int)
test_mni_df_labels_fake = np.ones(len(mni_df_test_fake), dtype=int)
test_mni_f2f_labels_fake = np.ones(len(mni_f2f_test_fake), dtype=int)
test_mni_fshifter_labels_fake = np.ones(len(mni_fshifter_test_fake), dtype=int)
test_mni_fswap_labels_fake = np.ones(len(mni_fswap_test_fake), dtype=int)
test_mni_nt_labels_fake = np.ones(len(mni_nt_test_fake), dtype=int)


val_ori_actor_labels_real = np.zeros(len(val_hog_ori_actor_real), dtype=int)
val_ori_youtube_labels_real = np.zeros(len(val_hog_ori_youtube_real), dtype=int)
val_mni_dfd_labels_fake = np.ones(len(val_hog_mni_dfd_fake), dtype=int)
val_mni_df_labels_fake = np.ones(len(val_hog_mni_df_fake), dtype=int)
val_mni_f2f_labels_fake = np.ones(len(val_hog_mni_f2f_fake), dtype=int)
val_mni_fshifter_labels_fake = np.ones(len(val_hog_mni_fshifter_fake), dtype=int)
val_mni_fswap_labels_fake = np.ones(len(val_hog_mni_fswap_fake), dtype=int)
val_mni_nt_labels_fake = np.ones(len(val_hog_mni_nt_fake), dtype=int)
##################################################################################################################

# Concatenate all training datasets into a single `train` variable
train_ff = np.concatenate([train_hog_ori_actor_real, train_hog_ori_youtube_real,train_hog_mni_dfd_fake,train_hog_mni_df_fake,
                            train_hog_mni_f2f_fake,train_hog_mni_fshifter_fake,train_hog_mni_fswap_fake,train_hog_mni_nt_fake], axis=0)

train_labels_ff = np.concatenate([train_ori_actor_labels_real, train_ori_youtube_labels_real,train_mni_dfd_labels_fake,train_mni_df_labels_fake,
                            train_mni_f2f_labels_fake,train_mni_fshifter_labels_fake,train_mni_fswap_labels_fake,train_mni_nt_labels_fake], axis=0)

# Concatenate all validation datasets into a single `val` variable
val_ff = np.concatenate([val_hog_ori_actor_real, val_hog_ori_youtube_real, val_hog_mni_dfd_fake, val_hog_mni_df_fake,
                            val_hog_mni_f2f_fake, val_hog_mni_fshifter_fake, val_hog_mni_fswap_fake, val_hog_mni_nt_fake], axis=0)
val_labels_ff = np.concatenate([val_ori_actor_labels_real, val_ori_youtube_labels_real, val_mni_dfd_labels_fake, val_mni_df_labels_fake,
                            val_mni_f2f_labels_fake, val_mni_fshifter_labels_fake, val_mni_fswap_labels_fake, val_mni_nt_labels_fake], axis=0)
# Concatenate all testing datasets into a single `test` variable
test_ff = np.concatenate([ori_actor_test_real, ori_youtube_test_real, mni_dfd_test_fake,
                           mni_df_test_fake, mni_f2f_test_fake, mni_fshifter_test_fake,
                           mni_fswap_test_fake, mni_nt_test_fake], axis=0)
test_labels_ff = np.concatenate([test_ori_actor_labels_real, test_ori_youtube_labels_real, test_mni_dfd_labels_fake, test_mni_df_labels_fake,
                            test_mni_f2f_labels_fake, test_mni_fshifter_labels_fake, test_mni_fswap_labels_fake, test_mni_nt_labels_fake], axis=0)


# Print the results for verification
# Print the results for verification
print(f"Total train: {len(train_ff)} images, and shape:{train_ff.shape}")
print(f"Total test: {len(test_ff)} images, and shape:{test_ff.shape}")
print(f"Total val: {len(val_ff)} images, and shape:{val_ff.shape}")


# Print results for verification
print(f"Train Labels: {len(train_labels_ff)} ")
print(f"Test Labels: {len(test_labels_ff)} ")
print(f"Val Labels: {len(val_labels_ff)} ")

Data loaded successfully from the HDF5 file.
Data loaded successfully from the HDF5 file.
Data loaded successfully from the HDF5 file.
Data loaded successfully from the HDF5 file.
Shape of the concatenated array: (2808, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (3299, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (2083, 160, 160, 3)
Shape of the concatenated array: (1742, 160, 160, 3)
Shape of the concatenated array: (1742, 160, 160, 3)
Training data ACTOR real shape: (1965, 160, 160, 3) Testing real data shape: (843, 160, 160, 3)
Training data Youtube real shape: (1458, 160, 160, 3) Testing real data shape: (625, 160, 160, 3)
Training data DFD fake shape: (2309, 160, 160, 3) Testing fake data shape: (990, 160, 160, 3)
Training data DF fake shape: (1458, 160, 160, 3) Testing fake data shape: (625, 160, 160, 3)
Training data f

In [11]:
# Convert data into PyTorch Dataset
train_dataset = ImageDataset(train_ff, train_labels_ff, transform)
val_dataset = ImageDataset(val_ff, val_labels_ff, transform)
test_dataset = ImageDataset(test_ff, test_labels_ff, transform)

# Dataloaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [15]:
# Train the model
epochs = 10  # Increase epochs for better accuracy
train_model(model, train_loader, val_loader, epochs)

Epoch 1/10: 100%|██████████| 706/706 [4:54:55<00:00, 25.06s/it]  


Epoch 1: LR = 3.40e-05, Loss = 0.6006, Acc = 70.92%
Val Metrics: {'accuracy': 0.7285031847133758, 'precision': 0.6868948531749695, 'recall': 0.7285031847133758, 'f1': 0.6241261778774199}



Epoch 2/10: 100%|██████████| 706/706 [5:02:47<00:00, 25.73s/it]  


Epoch 2: LR = 6.70e-05, Loss = 0.5022, Acc = 76.38%
Val Metrics: {'accuracy': 0.7921974522292994, 'precision': 0.7827881904563015, 'recall': 0.7921974522292994, 'f1': 0.7851557910766196}



Epoch 3/10: 100%|██████████| 706/706 [5:03:04<00:00, 25.76s/it]  
c:\Users\aneek\anaconda3\envs\tf_gpu_env\lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


...

Epoch 8: LR = 1.88e-05, Loss = 0.0222, Acc = 99.25%
Val Metrics: {'accuracy': 0.8757961783439491, 'precision': 0.873217455737871, 'recall': 0.8757961783439491, 'f1': 0.8713516754984924}



Epoch 9/10: 100%|██████████| 706/706 [4:54:29<00:00, 25.03s/it]  


Epoch 9: LR = 4.95e-06, Loss = 0.0051, Acc = 99.85%
Val Metrics: {'accuracy': 0.8757961783439491, 'precision': 0.8730582605809842, 'recall': 0.8757961783439491, 'f1': 0.8716651597754506}



Epoch 10/10: 100%|██████████| 706/706 [4:32:06<00:00, 23.12s/it]  


Epoch 10: LR = 0.00e+00, Loss = 0.0024, Acc = 99.96%
Val Metrics: {'accuracy': 0.875, 'precision': 0.8725734309708305, 'recall': 0.875, 'f1': 0.8732183370461996}



In [13]:
print("=== DATA LOADING ===")
start = monitor1.get_stats()
# Final evaluation on test data
test_metrics = evaluate_metrics(model, test_loader)
# Your data loading operations here
time.sleep(2)  # Simulate loading time
#7.7 gpu

end = monitor1.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")
print(f"Time Usage: {duration:.1f} s")
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")#7.7

=== DATA LOADING ===

=== RESOURCE USAGE ===
CPU Usage: 1.5%
Time Usage: 477.2 s
GPU Memory Used: 929.1 MB


In [16]:
# Final evaluation on test data
test_metrics = evaluate_metrics(model, test_loader)
print(f"Test Metrics:")
print(f"- Accuracy: {test_metrics['accuracy']:.4f}")
print(f"- Precision: {test_metrics['precision']:.4f}")
print(f"- Recall: {test_metrics['recall']:.4f}")
print(f"- F1 Score: {test_metrics['f1']:.4f}")

Test Metrics:
- Accuracy: 0.8751
- Precision: 0.8728
- Recall: 0.8751
- F1 Score: 0.8735


In [ ]:
# Your data loading operations here
time.sleep(2)  # Simulate loading time

end = monitor.get_stats()
duration = end['timestamp'] - start['timestamp']

print("\n=== RESOURCE USAGE ===")
print(f"CPU Usage: {end['cpu_%']:.1f}%")#15%
print(f"RAM Used: {end['ram_mb'] - start['ram_mb']:.1f} MB")#29GB
print(f"Time Usage: {duration:.1f} s")#3143m+52m 191700s
print(f"GPU Memory Used: {end['gpu_mem_mb']:.1f} MB")#7.6
print(f"Power Consumption: {int(end['power_w'])}W")  # Rounded to whole watts

NameError: name 'start' is not defined

In [24]:
test_dataset = ImageDataset(test_hog, test_labels, transform)
# Dataloaders
batch_size = 16
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f"Test Accuracy of DFC on F++ ON CrossVit")
# Final evaluation on test data
test_metrics = evaluate_metrics(model, test_loader)
print(f"Test Metrics:")
print(f"- Accuracy: {test_metrics['accuracy']:.4f}")
print(f"- Precision: {test_metrics['precision']:.4f}")
print(f"- Recall: {test_metrics['recall']:.4f}")
print(f"- F1 Score: {test_metrics['f1']:.4f}")


Test Accuracy of DFC on F++ ON CrossVit
Test Metrics:
- Accuracy: 0.5207
- Precision: 0.5212
- Recall: 0.5207
- F1 Score: 0.5176


In [22]:

test_dataset = ImageDataset(test, test_labels, transform)
# Dataloaders
batch_size = 16
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f"Test Accuracy of Celeb-DF(V2) on F++ ON CrossVit")
# Final evaluation on test data
test_metrics = evaluate_metrics(model, test_loader)
print(f"Test Metrics:")
print(f"- Accuracy: {test_metrics['accuracy']:.4f}")
print(f"- Precision: {test_metrics['precision']:.4f}")
print(f"- Recall: {test_metrics['recall']:.4f}")
print(f"- F1 Score: {test_metrics['f1']:.4f}")

Test Accuracy of Celeb-DF(V2) on F++ ON CrossVit
Test Metrics:
- Accuracy: 0.7830
- Precision: 0.7704
- Recall: 0.7830
- F1 Score: 0.7564
